In [1]:
import os
import logging
import pickle
from collections import defaultdict
from random import choices
import sys
from multiprocessing import Pool, cpu_count

# Set up logging
logging.basicConfig(filename='markov_model.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define a named function instead of lambda
def default_dict():
    return defaultdict(int)

# Global model state
markov_model = defaultdict(default_dict)

model_file = r'D:\Osama Khalid\Osama Khalid BSCY 7\FYP Part 1\AI Codes\ML_Algorithms\Markov_Model\markov_model.pkl'

# Read passwords from a text file
def read_passwords(file_path, encoding='utf-8'):
    if not os.path.exists(file_path):
        logging.error(f"Error: Password file not found at {file_path}")
        return []
    with open(file_path, 'r', encoding=encoding) as file:
        passwords = file.read().splitlines()
    return passwords

# Build Markov model
def build_markov_model(passwords, order=2):
    local_model = defaultdict(default_dict)
    for password in passwords:
        for i in range(len(password) - order):
            seq = password[i:i + order]
            next_char = password[i + order]
            local_model[seq][next_char] += 1
    return local_model

# Combine local models into the global model
def combine_models(global_model, local_model):
    for seq, next_chars in local_model.items():
        for next_char, count in next_chars.items():
            global_model[seq][next_char] += count

# Generate password using Markov model
def generate_password(model, length=8, order=2):
    if not model:
        logging.error("Error: Markov model is empty")
        return ""
    
    start_seq = choices(list(model.keys()))[0]
    result = start_seq
    for _ in range(length - order):
        last_seq = result[-order:]
        next_char_options = list(model[last_seq].keys())
        if not next_char_options:
            logging.error(f"Error: No next character found for sequence {last_seq}")
            return result
        next_char = choices(next_char_options, list(model[last_seq].values()))[0]
        result += next_char
    return result

# Save model state
def save_model_state():
    with open(model_file, 'wb') as file:
        pickle.dump(dict(markov_model), file)
    logging.info("Model state saved successfully.")
    print("Model state saved successfully.")

# Load model state
def load_model_state():
    global markov_model
    if os.path.exists(model_file):
        try:
            with open(model_file, 'rb') as file:
                markov_model.update(pickle.load(file))
            logging.info("Model state loaded successfully.")
            print("Model state loaded successfully.")
            return True
        except EOFError:
            logging.error("Error: End of file reached unexpectedly during unpickling.")
            return False
        except Exception as e:
            logging.error(f"Error occurred while loading model state: {e}")
            return False
    else:
        logging.error("Error: Model file not found.")
        return False

# Process a single file
def process_file(file_path, order=2):
    print(f"Processing file: {file_path}")
    passwords = read_passwords(file_path)
    if not passwords:
        logging.error(f"Error: No passwords read from the file {file_path}.")
        return defaultdict(default_dict)
    local_model = build_markov_model(passwords, order)
    logging.info(f"File {file_path} processed.")
    print(f"File {file_path} processed.")
    return local_model

# Main execution
def main(directory_path, order=2, password_length=8, num_passwords=10):
    if not os.path.exists(directory_path):
        logging.error(f"Error: Directory not found at {directory_path}")
        print(f"Error: Directory not found at {directory_path}")
        sys.exit(1)
    
    if not load_model_state():
        logging.info("Starting with a new model.")
        print("Starting with a new model.")
    
    # Get list of files in the directory
    files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
    
    # Process files using multiprocessing
    with Pool(cpu_count()) as pool:
        local_models = pool.map(process_file, files)
    
    # Combine all local models into the global model
    for local_model in local_models:
        combine_models(markov_model, local_model)
    
    save_model_state()

    generated_passwords = [generate_password(markov_model, password_length, order) for _ in range(num_passwords)]
    
    print("Generated passwords:")
    for pwd in generated_passwords:
        print(pwd)
        logging.info(f"Generated password: {pwd}")

if __name__ == "__main__":
    input_directory = r'D:\Osama Khalid\Osama Khalid BSCY 7\FYP Part 1\AI Codes\Filtered_Passwords_Dataset\length_6'  # Change this to your directory path
    main(input_directory)


Starting with a new model.


new code